In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import MNIST
from torchvision.utils import make_grid
from torchvision.transforms import ToTensor
from torch.utils.data import random_split
from torch.utils.data.dataloader import DataLoader
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

In [ ]:
data=MNIST(download=True,root='data/',transform=ToTensor())

In [ ]:
train,val=random_split(data,[50000,10000])

In [ ]:
batch_size=200
train_loader=DataLoader(train,batch_size,shuffle=True,num_workers=4,pin_memory=True)
val_loader=DataLoader(val,batch_size,num_workers=4,pin_memory=True)

In [ ]:
for xb,yb in train_loader:
    print(xb.shape)
    plt.figure(figsize=(16,8))
    plt.axis('off')
    plt.imshow(make_grid(xb,nrow=20).permute(1,2,0))
    break


In [ ]:
class Neural(nn.Module):
    def __init__(self,in_size,hidden_size,out_size):
        super().__init__()
        #hidden layer
        self.linear1=nn.Linear(in_size,hidden_size)
        #output layer
        self.linear2=nn.Linear(hidden_size,out_size)
        
    def forward(self,xb):
        # Flatten the image tensors
        xb=xb.view(xb.size(0),-1)
        # Get intermediate outputs using hidden layer
        out=self.linear1(xb)
        # Apply activation function
        out=F.relu(out)
        # Get predictions using output layer
        out=self.linear2(out)
        return out
    
    def training_step(self,batch):
        xb,yb =batch
        out=self(xb)
        loss=F.cross_entropy(out,yb)
        return loss
    
    def validation_step(self, batch):
        xb,yb=batch
        out=self(xb)
        loss=F.cross_entropy(out,yb)
        acc=accuracy(out,yb)
        return {'val_loss':loss.detach(),'val_acc':acc}
    
    def validation_step_end(self ,outputs):#thisfunction is used to fing the overall mean of all batches loss and accuracy
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
        
    def epoch_end(self,epoch,result):
        print("epoch[{}],val_loss:{:.4f}.val_acc:{:.4f}".format(epoch,result['val_loss'],result['val_acc']))
    

In [ ]:
#function to calculate accuracy
def accuracy(out,yb):
    _,pred=torch.max(out,dim=1)
    return torch.tensor(torch.sum(pred==yb).item()/len(pred))


In [ ]:
in_size = 784
hidden_size = 32 # you can change this
out_size = 10

model=Neural(in_size,hidden_size,out_size)

In [ ]:
for xb,yb in train_loader:
    out=model(xb)
    loss=F.cross_entropy(out,yb)
    print('loss',loss.item())
    break

In [ ]:
torch.cuda.is_available()

In [ ]:
def get_default_device():
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        torch.device('cpu')

In [ ]:
device=get_default_device()
device

In [ ]:
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

In [ ]:
for images, labels in train_loader:
    print(images.shape)
    images = to_device(images, device)
    print(images.device)
    break

In [ ]:
class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [ ]:
train_loader=DeviceDataLoader(train_loader,device)
val_loader=DeviceDataLoader(val_loader,device)

In [ ]:
for xb,yb in train_loader:
    print('xb.device:', xb.device)
    print('yb:', yb)
    break

In [ ]:
def evaluate(model,val_loader):
    outputs=[model.validation_step(batch) for batch in val_loader]
    return model.validation_step_end(outputs)

In [ ]:
def fit(epochs,model,lr,train_loader,val_loader,opt_func=torch.optim.SGD):
    history=[]
    optimizer=opt_func(model.parameters(),lr)
    for epoch in range(epochs):
        #training phase
        for batch in train_loader:
            loss=model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        #validation phase
        result=evaluate(model,val_loader)
        model.epoch_end(epoch,result)
        history.append(result)
    return history

In [ ]:
#move model to gpu
model=Neural(in_size,hidden_size,out_size)
to_device(model ,device)

In [ ]:
history=[evaluate(model,val_loader)]
history

In [ ]:
history+=fit(5,model,0.5,train_loader,val_loader)

In [ ]:
history+=fit(5,model,0.1,train_loader,val_loader)

In [ ]:
losses = [x['val_loss'] for x in history]
plt.plot(losses, '-x')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.title('Loss vs. No. of epochs');

In [ ]:
accuracies = [x['val_acc'] for x in history]
plt.plot(accuracies, '-x')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.title('Accuracy vs. No. of epochs');

In [ ]:
test_dataset = MNIST(root='data/', train=False, transform=ToTensor())
test_loader=DataLoader(test_dataset,batch_size,num_workers=True,pin_memory=True)
test_loader=DeviceDataLoader(test_loader,device)

In [ ]:
#accuracy of test dataset
evaluate(model,test_loader)

In [ ]:
def predict_image(img, model):
    xb = to_device(img.unsqueeze(0), device)
    yb = model(xb)
    _, preds  = torch.max(yb, dim=1)
    return preds[0].item()

In [ ]:
img, label = test_dataset[1000]
plt.imshow(img[0], cmap='gray')
print('Label:', data.classes[label], ', Predicted:', data.classes[predict_image(img, model)])

In [ ]:
img, label = test_dataset[0]
plt.imshow(img[0], cmap='gray')
print('Label:', data.classes[label], ', Predicted:', data.classes[predict_image(img, model)])

In [ ]:
img, label = test_dataset[9999]
plt.imshow(img[0], cmap='gray')
print('Label:', data.classes[label], ', Predicted:', data.classes[predict_image(img, model)])